In [1]:
import quandl
import pandas as pd
import numpy as np
from xgboost.sklearn import XGBRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score,roc_auc_score
from sklearn.model_selection import GridSearchCV, train_test_split,RandomizedSearchCV #Hiperparametrización
quandl.ApiConfig.api_key = 'DEN8cmx6MYrtzyeUdDeh' #????

In [2]:
%%time
tickers = ['FB','GOOGL','AAPl','AMZN','TSLA','MSFT','AMD','INTC','T','NFLX']
Validate = [] #Conjunto para validación 
Train = [] #Conjunto de entrenamiento

for ticker in tickers:
    df = quandl.get("WIKI/"+ticker)
    #print (df)
    df = df.iloc[-500:]
    func = ['mean','std','skew','kurt'] #Estadísticos de interés
    time = [2,4,7,30,60,120,250] #Días tomados artibatriamente  
    Z = pd.DataFrame()  
    for f in func: 
        for t in time:
            aux = pd.DataFrame()
            for i in range(249):
                aux[ df.index[i+251] ] = list( df.iloc[i-t+251:i+251]['Adj. Close'] ) #
            Z[f+str(t)] = aux.apply(f,axis=0)  
    Z.drop(['skew2','kurt2'],axis=1, inplace =True)
    W = pd.merge(Z, df[['Adj. Close']],left_index=True,right_index=True)    
    X = W.drop(['Adj. Close'],1).fillna(0)
    y = W[['Adj. Close']]    
    mm1 = MinMaxScaler()
    mm2 = MinMaxScaler()
    mm1.fit(X)
    mm2.fit(y)
    Xm = pd.DataFrame(mm1.transform(X))
    ym = pd.DataFrame(mm2.transform(y))
    Xt, Xv, yt, yv = train_test_split(Xm, ym, test_size = 0.3)
    model = XGBRegressor()
    model.fit(Xt,yt)   
    print ticker
    print 'ACC Validate {:.2%}'.format(model.score(Xv,yv))
    print 'ACC Train    {:.2%}'.format(model.score(Xt,yt))
    Validate.append(model.score(Xv,yv))
    Train.append(model.score(Xt,yt))
print '\n'
print 'Promedio Validate  {:.2%}'.format(np.array(Validate).mean())
print 'Promedio Train     {:.2%}'.format(np.array(Train).mean())

FB
ACC Validate 95.29%
ACC Train    99.77%
GOOGL
ACC Validate 96.37%
ACC Train    99.77%
AAPl
ACC Validate 95.82%
ACC Train    99.80%
AMZN
ACC Validate 99.19%
ACC Train    99.95%
TSLA
ACC Validate 82.88%
ACC Train    98.97%
MSFT
ACC Validate 97.88%
ACC Train    99.95%
AMD
ACC Validate 71.46%
ACC Train    99.32%
INTC
ACC Validate 98.56%
ACC Train    99.91%
T
ACC Validate 84.68%
ACC Train    99.26%
NFLX
ACC Validate 99.14%
ACC Train    99.94%


Promedio Validate  92.13%
Promedio Train     99.67%
CPU times: user 2min 57s, sys: 1.2 s, total: 2min 58s
Wall time: 3min 24s


Observación: Los datos que estaban dando como Target son continuos entonces se debía cambiar la SVM en modo regresión 